# Medindo retorno

Já treinamos nosso modelo, avaliamos dentre as opções qual melhor se saiu. Agora é hora de mensurar a expectativa de retorno. Para isso vamos simular 2 estratégias:

* Estratégia 1 (Modelo de ML): Para cada cliente, o modelo previu a probabilidade de aceite para todas as ofertas disponíveis e selecionou apenas a oferta com a maior probabilidade.

* Estratégia 2 (Aleatória): Para cada cliente, uma oferta foi sorteada ao acaso.

Para cada estratégia, vamos buscar no histórico de dados o valor que os clientes realmente gastaram (`valor_gasto_na_jornada`) quando receberam a oferta selecionada. O "Impacto Financeiro" que será mostrado no gráfico é a soma total desses valores para cada uma das duas estratégias.

In [0]:
import mlflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
PATH_BASE_PROCESSADA = 'workspace.processed.base_processada'
PATH_MODELO = "models:/workspace.modelos_ml.ModeloOferta_RandomForest@producao"

In [0]:
print(f"Carregando modelo: {PATH_MODELO}")
load_model = mlflow.sklearn.load_model(PATH_MODELO)
print("Modelo carregado!")

In [0]:
dados = spark.table(PATH_BASE_PROCESSADA).toPandas()

In [0]:
# Lista de colunas a descartar
cols_drop = [
    "account_id", "offer_id_final", "offer_received", "offer_viewed",
    "valor_gasto_na_jornada", "qtd_transacoes_na_jornada",
    "ticket_medio_na_jornada", "registered_on", "channels"
]

# Separação entre features e target
X = dados.drop(columns=["target"], errors="ignore")
y = dados["target"]

# Removendo colunas não desejadas
X = X.drop(columns=cols_drop, errors="ignore")


In [0]:
customer_features = ['account_id', 'age', 'credit_card_limit', 'gender', 'tempo_de_registro', 'receive_time']
offer_features = [
    'offer_id_final', 'discount_value', 'duration', 'min_value', 'offer_type',
    'categoria_duracao', 'num_channels', 'email', 'mobile', 'social', 'web'
]

In [0]:
# Obter ofertas únicas
dados_ofertas = dados.drop_duplicates(subset='offer_id_final')[offer_features].reset_index(drop=True)

# Selecionar uma amostra de clientes únicos para a simulação
clientes_simulacao = dados.drop_duplicates(subset='account_id')[customer_features].reset_index(drop=True)

In [0]:
resultados_modelo = []
for _, cliente in clientes_simulacao.iterrows():
    df_cliente_unic = pd.DataFrame([cliente.drop('account_id').to_dict()])
    df_scoring = df_cliente_unic.merge(dados_ofertas.drop('offer_id_final', axis=1), how='cross')
    
    probabilidades = load_model.predict_proba(df_scoring)[:, 1]
    melhor_oferta_idx = np.argmax(probabilidades)
    oferta_escolhida_id = dados_ofertas.iloc[melhor_oferta_idx]['offer_id_final']
    
    resultado_real = dados[(dados['account_id'] == cliente['account_id']) & (dados['offer_id_final'] == oferta_escolhida_id)]
    if not resultado_real.empty:
        resultados_modelo.append(resultado_real.iloc[0]['valor_gasto_na_jornada'])

In [0]:
resultados_aleatorio = []
for _, cliente in clientes_simulacao.iterrows():
    oferta_aleatoria_id = dados_ofertas.sample(1).iloc[0]['offer_id_final']
    resultado_real = dados[(dados['account_id'] == cliente['account_id']) & (dados['offer_id_final'] == oferta_aleatoria_id)]
    if not resultado_real.empty:
        resultados_aleatorio.append(resultado_real.iloc[0]['valor_gasto_na_jornada'])


In [0]:
# Calcular o impacto total
impacto_modelo = np.sum(resultados_modelo)
impacto_aleatorio = np.sum(resultados_aleatorio)


In [0]:
print(f"Modelo: {impacto_modelo}")
print(f"Aleatório: {impacto_aleatorio}")

In [0]:
minhas_cores = [
    '#FE3054', # vermelho vibrante
    '#F94EB6', # rosa choque
    '#FFB020', # amarelo ouro
    '#FA781A', # laranja intenso
    '#FEF6C8', # bege claro
    '#06516E', # azul petróleo
    '#06987B', # verde esmeralda
    '#260607'  # vinho quase preto
]

minha_paleta = sns.color_palette(minhas_cores)

sns.palplot(minha_paleta)
plt.title("Minha Paleta Personalizada")
plt.show()

sns.set_palette(minha_paleta)

In [0]:
# Preparar dados para o gráfico
dados_grafico = pd.DataFrame({
    'Estratégia': ['Modelo de ML', 'Aleatória'],
    'Impacto Financeiro (R$)': [impacto_modelo, impacto_aleatorio]
}).sort_values('Impacto Financeiro (R$)', ascending=False)

plt.figure(figsize=(10, 6))
barplot = sns.barplot(x='Estratégia', y='Impacto Financeiro (R$)', data=dados_grafico, hue='Estratégia')
for p in barplot.patches:
    barplot.annotate(f'R$ {p.get_height():,.2f}', 
                     (p.get_x() + p.get_width() / 2., p.get_height()), 
                     ha='center', va='center', 
                     xytext=(0, 9), 
                     textcoords='offset points')

plt.title('Comparativo de Impacto Financeiro por Estratégia de Envio de Oferta', fontsize=16)
plt.xlabel('Estratégia', fontsize=12)
plt.ylabel('Valor Gasto Total Simulado (R$)', fontsize=12)
plt.tight_layout()
plt.savefig('impacto_financeiro.png')


O gráfico mostra que a personalização impulsionada por Machine Learning capaz de identificar a melhor oferta para determinado usuário/cliente gera um valor de negócio muito superior à abordagem aleatória. A capacidade do modelo de discernir qual oferta é mais adequada para cada perfil de cliente resultou em um impacto financeiro simulado 38% maior que e distribuição aleatória, justificando sua implementação.